In [ ]:
import timm
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
from timm.data import resolve_model_data_config, create_transform
import random
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from common.datasets import LandmarkDataset
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from tqdm import tqdm

##### Environment variables

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATASET_PATH = "D:/Datasets/landmark-recognition-2021"
TRAIN_DIR = f"{DATASET_PATH}/train"
TEST_DIR = f"{DATASET_PATH}/train"

##### Hyperparameters

In [ ]:
SEED = 42
EPOCHS = 1
BATCH_SIZE = 32
N_WORKERS = 6

##### Training IDs table

In [ ]:
train_df = pd.read_csv(f"{DATASET_PATH}/train.csv")
train_df.head()

##### Seeding

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # for multi-GPU setups

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(SEED)

##### Training/Test datasets

In [ ]:
N_CLASSES = len(train_df["landmark_id"].value_counts())

X = train_df.drop("landmark_id", axis=1)
y = train_df["landmark_id"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
model = timm.create_model("efficientnetv2_s", num_classes=N_CLASSES).to(DEVICE)
data_config = timm.data.resolve_model_data_config(model)
transform = timm.data.create_transform(**data_config)

In [ ]:
use_subset = True

train_dataset = LandmarkDataset(X_train, y_train, transform=transform, directory=TRAIN_DIR)
test_dataset = LandmarkDataset(X_test, y_test, transform=transform, directory=TRAIN_DIR)

if use_subset:
    train_indices = np.random.choice(len(train_dataset), 64, replace=False)
    test_indices = np.random.choice(len(test_dataset), 64, replace=False)

    train_dataset = Subset(train_dataset, train_indices)
    test_dataset = Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=N_WORKERS, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=N_WORKERS, shuffle=True, pin_memory=True)

len(train_dataset), len(test_dataset)

##### Training loop

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_bar = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in train_bar:
        images = images.to(DEVICE, dtype=torch.float)
        labels = labels.to(DEVICE, dtype=torch.long)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        train_bar.set_postfix({
            "loss": f"{running_loss/total:.4f}",
            "acc": f"{100*correct/total:.2f}%"
        })

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100.0 * correct / total
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    val_bar = tqdm(test_loader, desc="Validation", leave=False)
    with torch.no_grad():
        for images, labels in val_bar:
            images = images.to(DEVICE, dtype=torch.float)
            labels = labels.to(DEVICE, dtype=torch.long)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_bar.set_postfix({
                "val_loss": f"{val_loss/total:.4f}",
                "val_acc": f"{100*correct/total:.2f}%"
            })

    val_loss = val_loss / len(test_loader.dataset)
    val_acc = 100.0 * correct / total
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")